In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
pd.set_option('display.max_columns',None)
DATA_DIR = Path('../data')
NYC_COUNTIES = ['New York County']
pandaload = lambda x: pd.read_csv(Path.joinpath(DATA_DIR,x),index_col=0)
keep = [
    'trtid',
    'pop',
    'nhwht',
    'nhblk',
    'hisp',
    'asian',
    'fb',
    'nimm',
    'nat',
    'olang',
    'lep',
    'hs',
    'col',
    'unemp',
    'flabf',
    'prof',
    'manuf',
    'semp',
    'vet',
    'npov',
    'n65pov',
    'nfmpov',
    'vac',
    'own',
    'multi',
    'h30old',
    'a18und',
    'a60up',
    'a75up',
    'wds',
    'fhh',
    'incpc'
]

In [2]:
p = Path('../data/LTDB_2000.csv')
if p.exists():
    data2000 = pd.read_csv(p)
else:
    data2000f = pd.read_excel('../data/LTDB_Full_2000.xlsx')
    data2000f = data2000f[data2000f.state == 'NY'][data2000f.county.isin(NYC_COUNTIES)]
    data2000s = pd.read_excel('../data/LTDB_Sample_2000.xlsx')
    data2000s = data2000s[data2000s.state == 'NY'][data2000s.county.isin(NYC_COUNTIES)]

    data2000 = data2000f.merge(data2000s, how='inner')
    del data2000f, data2000s
    data2000.to_csv(p)
    
p = Path('../data/LTDB_2010.csv')
if p.exists():
    data2010 = pd.read_csv(p)
else:
    data2010f = pd.read_excel('../data/LTDB_Full_2010.xlsx')
    data2010f = data2010f[data2010f.state == 'NY'][data2010f.county.isin(NYC_COUNTIES)]

    data2010s = pd.read_excel('../data/LTDB_Sample_2010.xlsx')
    data2010s = data2010s[data2010s.tractid.isin(data2010f.tractid.values)]

    data2010 = data2010f.merge(data2010s, how='inner')
    del data2010f, data2010s
    data2010.to_csv(p)
for column in data2010.columns:
    if isinstance(column,str):
        if '12' in column and (column.replace('12','10') in data2010.columns or column.replace('12','08') in data2010.columns):
            print(f'Dropping {column}')
            data2010 = data2010.drop(column,axis=1)
columns2000 = list(data2000.columns)
columns2010 = list(data2010.columns)
replace = lambda x: x.replace('90','').replace('00','').replace('10','').replace('12','').lower()
columns2000 = {c:replace(c) for c in columns2000 if isinstance(c,str)}
columns2010 = {c:replace(c) for c in columns2010 if isinstance(c,str)}
columns2010['tractid'] = 'trtid'
data2000 = data2000.rename(columns=columns2000)
data2010 = data2010.rename(columns=columns2010)
assert(len(set(data2010.columns)) == len(list(data2010.columns)))
columns = set(data2000.columns) & set(data2010.columns)
data2000 = data2000[columns]
data2010 = data2010[columns]


Dropping ageblk12
Dropping agentv12
Dropping agewht12
Dropping agehsp12
Dropping india12
Dropping filip12
Dropping japan12
Dropping korea12
Dropping viet12
Dropping pop12
Dropping nhwht12
Dropping nhblk12
Dropping ntv12
Dropping hisp12
Dropping asian12
Dropping haw12
Dropping china12
Dropping a15wht12
Dropping a15blk12
Dropping a15hsp12
Dropping a15ntv12
Dropping ageasn12
Dropping a15asn12
Dropping mex12
Dropping pr12
Dropping cuban12
Dropping family12
Dropping hu12
Dropping vac12
Dropping ohu12
Dropping own12
Dropping rent12
Dropping a18und12
Dropping a60up12
Dropping a75up12
Dropping fhh12


In [3]:
data2000 = data2000[keep]
data2010 = data2010[keep]

In [4]:
original = pd.read_csv('../data/NY_2000.csv')

In [5]:
print(len(data2000))
print(len(data2010))

287
288


In [6]:
trt00 = set(data2000.trtid.values)
trt10 = set(data2010.trtid.values)
print(f'In 00 but not 10: {trt00 - trt10}')
print(f'In 10 but not 00: {trt10 - trt00}')
display(data2010[data2010.trtid.isin(trt10 - trt00)])

data2010 = data2010[data2010.trtid.isin(trt00)]
data2000 = data2000[data2000.trtid.isin(trt10)]

In 00 but not 10: set()
In 10 but not 00: {36061000100}


,trtid,pop,nhwht,nhblk,hisp,asian,fb,nimm,nat,olang,lep,hs,col,unemp,flabf,prof,manuf,semp,vet,npov,n65pov,nfmpov,vac,own,multi,h30old,a18und,a60up,a75up,wds,fhh,incpc
0,36061000100,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-999


In [7]:
print(len(data2000))
print(len(data2010))

287
287


In [8]:
data2000.to_csv(Path.joinpath(DATA_DIR,'data2000.csv'))
data2010.to_csv(Path.joinpath(DATA_DIR,'data2010.csv'))

In [9]:
assert(np.all(data2000.columns == data2010.columns))

In [10]:
data2010 = data2010.set_index('trtid')
data2000 = data2000.set_index('trtid')

In [11]:
total = 0
for k,v in np.sum(data2010 == -999).iteritems():
    if v > 0:
        print(k,v)
        total += v
print(total)

incpc 7
7


In [12]:
remove = data2010[(data2010 == -999).any(axis=1)].index.values

In [13]:
data2000 = data2000[~data2000.index.isin(remove)]
data2010 = data2010[~data2010.index.isin(remove)]

In [28]:
data = data2010/data2000

In [29]:
data = data.replace([np.inf, -np.inf], np.nan)

In [30]:
total = 0
for k,v in np.sum(data.isna()).iteritems():
    if v>len(data)//10:
        print(k,v)
        total += v
        data = data.drop(k,axis=1)

nfmpov 40


In [31]:
data = data[~data.isna().any(axis=1)]

In [32]:
len(data)

253

In [33]:
data = np.log(data)

/home/alex/anaconda3/envs/isabelle/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [34]:
data = data.replace([np.inf, -np.inf], np.nan)

In [35]:
total = 0
for k,v in np.sum(data.isna()).iteritems():
    if v>len(data)//10:
        print(k,v)
        total += v
        data = data.drop(k,axis=1)

In [36]:
data = data.dropna()

In [37]:
len(data)

236

In [38]:
mus = data.mean()
sis = data.std()

data = (data - mus) / sis

In [39]:
len(data)

236

In [40]:
len(data.columns)

30

In [41]:
data.to_csv(Path.joinpath(DATA_DIR,'NY_2000_NEW.csv'))